То что советуют на Habr

In [ ]:
""" Библиотеки для предобработки картинок  """ 
from glob import glob  # Позволит читать пути до папок 
import cv2 as cv
import re
import numpy as np

In [ ]:
""" Cоздадим скриптом по 9 папок (каждая соответствует одному символу) внутри training_data и test_data. 
В них мы и будем хранить картинки с вырезанными областями из картинки с капчей, соответствующими одному символу:"""
target_mapper = {'1':1 , '2':2, '3':3 , '4':4, '5':5, '6':6, '7':7 , '8':8, '9':9, '0':0}
parent_dir = os.getcwd()

train_data = os.path.join(parent_dir, "training_data")
test_data = os.path.join(parent_dir, "test_data")

for symbol in target_mapper.keys():
    path = os.path.join(train_data, str(target_mapper[symbol]))
    try:
        os.mkdir(path)
    except:
        print(f"{path} already exists")

    path = os.path.join(test_data, str(target_mapper[symbol]))
    try:
        os.mkdir(path)
    except:
        print(f"{path} already exists")

In [ ]:
"""Считываем картинку и делаем ее бинарной """
img = cv.imread(all_captchas[idx])

# convert to grayscale
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)


In [ ]:
"""размажем немного картинку и создадим новый объект, где мы по-пиксельно разделим нашу картинку в серых тонах на заблюренную. 
Это поможет сразу в двух вещах: сделать единые контуры символов без разрывов и снизить уровень шума на картинках. (Вот этот момент я не понял до конца )"""
# blur
blur = cv.GaussianBlur(gray, (0, 0), sigmaX=33, sigmaY=33)
# divide
divide = cv.divide(gray, blur, scale=255)

In [ ]:
""" Применяем алгоритм OTSU для того чтобы пиксель имел значения только 0 или 255  """
# otsu threshold
thresh = cv.threshold(divide, 0, 255, cv.THRESH_BINARY+cv.THRESH_OTSU)[1] 

In [ ]:
""" Теперь проведем очистку шумов внутри наших символов, чтобы получить четкие контуры. 
Для того, чтобы сделать это, используются две операции: эрозия и расширение """
# apply morphology
kernel = cv.getStructuringElement(cv.MORPH_RECT, (4, 4)) # создание ядра
morph = cv.morphologyEx(thresh, cv.MORPH_CLOSE, kernel) # dilation & erosion

In [ ]:
""" Теперь проведем операцию, которая оставит нам только тонкую полоску на границе символа без заливки 
(не обязательно, но может помочь оставить меньше внутренних контуров) """
threshold = 150
canny_output = cv.Canny(morph, threshold, threshold * 2)
canny_output_cp = canny_output.copy()
canny_output = cv.blur(canny_output, (3, 3))

In [ ]:
""" найдем контуры """
contours, hierarchy = cv.findContours(
  canny_output, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE
)

In [ ]:
То что посоветовал ChatGPT (по разбитию картинки на контуры (цифры) и их разделению) 

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

# Загрузим изображение
image_path = '/mnt/data/image.png'
image = cv2.imread(image_path)

# Преобразуем изображение в оттенки серого
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Применим бинарную пороговую обработку
_, binary = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# Найдем контуры
contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Отсортируем контуры слева направо
contours = sorted(contours, key=lambda c: cv2.boundingRect(c)[0])

# Нарисуем контуры на изображении
contoured_image = image.copy()
cv2.drawContours(contoured_image, contours, -1, (0, 255, 0), 2)

# Покажем изображение с контурами
plt.figure(figsize=(10, 5))
plt.imshow(cv2.cvtColor(contoured_image, cv2.COLOR_BGR2RGB))
plt.title('Контуры цифр')
plt.axis('off')
plt.show()

# Сохраним отдельные цифры как отдельные изображения
digit_images = []
for i, contour in enumerate(contours):
    x, y, w, h = cv2.boundingRect(contour)
    digit_image = image[y:y+h, x:x+w]
    digit_images.append(digit_image)
    cv2.imwrite(f'/mnt/data/digit_{i+1}.png', digit_image)

digit_images, [f'/mnt/data/digit_{i+1}.png' for i in range(len(digit_images))]
